In [108]:
#Import dependencies

import pandas as pd
import numpy as np
import glob
import datetime as dt
from datetime import timedelta
import time
import schedule
import csv
import xlsxwriter
import xlwt
from openpyxl.workbook import Workbook
from openpyxl.worksheet.properties import WorksheetProperties, PageSetupProperties
from pandas import ExcelWriter
from win32com.client import Dispatch
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 500)
from selenium.webdriver.common.action_chains import ActionChains
import shutil
from bs4 import BeautifulSoup
from config import password

In [112]:
options = webdriver.ChromeOptions()
profile = {
        'download.default_directory' : 'C:/Users/dshaf/Documents/Fantasy Baseball/',
        'download.prompt_for_download' : False,
        'download.extensions_to_open' : ''
}

options.add_experimental_option('prefs', profile)
browser = webdriver.Chrome(executable_path=r'C:/Users/dshaf/Documents/Fantasy Baseball/chromedriver.exe', options = options)
espn_url = 'http://www.espn.com/login'

browser.get(espn_url)
time.sleep(5)
iframe = browser.find_element_by_id('disneyid-iframe')
browser.switch_to.frame(iframe)
browser.find_element_by_xpath("//input[@type='email']").send_keys('dshaffer8641@gmail.com')
browser.find_element_by_xpath("//input[@type='password']").send_keys(password)
time.sleep(3)
browser.find_element_by_class_name('btn').click()
time.sleep(10)
browser.find_element_by_link_text('Fantasy').click()
time.sleep(3)
browser.find_element_by_xpath('//span[text()="Bryzzo Souvenir Co."]').click()
time.sleep(5)
browser.find_element_by_xpath('//span[text()="Scoreboard"]').click()

time.sleep(15)
soup = BeautifulSoup(browser.page_source, 'html.parser')
td = soup.select('td')
df = pd.DataFrame(columns=['Team','H','R','HR','TB','RBI','BB','K','SB','AVG','OPS',\
                           'IP','HRA','QS','W','L','SV','BAA','ERA','WHIP','K/9'])

lst = []

for txt in np.arange(0, len(td)):
    if not td[txt].text.startswith('BattingPitching'):
        lst.append(td[txt].text)

df['Team'] = lst[0::21]
df['H'] = lst[1::21]
df['R'] = lst[2::21]
df['HR'] = lst[3::21]
df['TB'] = lst[4::21]
df['RBI'] = lst[5::21]
df['BB'] = lst[6::21]
df['K'] = lst[7::21]
df['SB'] = lst[8::21]
df['AVG'] = lst[9::21]
df['OPS'] = lst[10::21]
df['IP'] = lst[11::21]
df['HRA'] = lst[12::21]
df['QS'] = lst[13::21]
df['W'] = lst[14::21]
df['L'] = lst[15::21]
df['SV'] = lst[16::21]
df['BAA'] = lst[17::21]
df['ERA'] = lst[18::21]
df['WHIP'] = lst[19::21]
df['K/9'] = lst[20::21]

display(df)

,Team,H,R,HR,TB,RBI,BB,K,SB,AVG,OPS,IP,HRA,QS,W,L,SV,BAA,ERA,WHIP,K/9
0,.com,1,1,0,2,0,0,1,0,.111,.333,1.1,0,0,0,0,0,.000,0.00,0.00,6.75
1,BRYZ,5,4,4,18,8,2,10,0,.208,1.009,6.0,1,0,0,0,0,.227,4.50,1.17,10.50
2,Jake,11,5,2,19,8,3,8,0,.314,.911,5.2,0,0,0,0,0,.118,0.00,0.35,6.35
3,C187,5,3,1,8,1,5,2,0,.294,.925,7.0,0,1,1,0,0,.091,0.00,0.57,11.57
4,DRDN,0,0,0,0,1,1,4,0,.000,.056,12.0,0,1,1,0,0,.159,0.75,0.67,14.25
5,CAP,9,5,3,20,4,3,8,1,.300,1.049,0.0,0,0,0,0,0,.000,0.00,0.00,0.00
6,OD,10,7,4,22,7,2,4,0,.313,1.076,4.2,2,0,0,1,0,.400,9.64,2.14,3.86
7,BYE,4,1,1,7,4,2,8,0,.133,.421,5.0,2,0,0,1,0,.455,12.60,2.20,12.60
8,PGGS,7,6,2,13,8,2,7,1,.250,.777,0.0,0,0,0,0,0,.000,0.00,0.00,0.00
9,FSU!,6,6,3,15,6,6,9,0,.240,.987,6.0,2,0,0,0,0,.280,6.00,1.17,12.00


In [ ]:
##### NOT PART OF THE PROGRAM #####

cols = ['Team','H','R','HR','TB','RBI','BB','K','SB','AVG','OPS',\
                            'IP','HRA','QS','W','L','SV','BAA','ERA','WHIP','K/9']
for i,j in enumerate(cols):